In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.3. Data Wrangling/Output'
cur_date = '081925'

library(readxl)
library(tidyverse)
library(reshape2)
library(lubridate)
library(oce)
library(tigris)

# reading in files
ohio_tmax_2020_df = read_csv("Input/Ohio_Tmax_2020.csv")
ohio_tmax_2021_df = read_csv('Input/Ohio_Tmax_2021.csv')
ohio_tmax_2022_df = read_csv('Input/Ohio_Tmax_2022.csv')
ohio_PM_2020_df = read_csv('Input/Ohio_PM_2020.csv')
ohio_PM_2021_df = read_csv('Input/Ohio_PM_2021.csv')
ohio_PM_2022_df = read_csv('Input/Ohio_PM_2022.csv')
ohio_tmin_2020_df = read_csv('Input/Ohio_Tmin_2020.csv')
ohio_tmin_2021_df = read_csv('Input/Ohio_Tmin_2021.csv')
ohio_tmin_2022_df = read_csv('Input/Ohio_Tmin_2022.csv')
ohio_HI_2020_df = read_csv('Input/Ohio_HI_2020.csv')
ohio_HI_2021_df = read_csv('Input/Ohio_HI_2021.csv')
ohio_HI_2022_df = read_csv('Input/Ohio_HI_2022.csv')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: gsw

To enable caching of data, set `options(tigris_use_cache = TRUE)`
in your R script or .Rprofile.

Rows: 1048575 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): DOY, x, y, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types 

In [2]:
head(ohio_tmax_2020_df)

DOY,x,y,Value
<dbl>,<dbl>,<dbl>,<dbl>
50,2032326,4815818,76.1
50,2033252,4815818,76.1
50,2030473,4814892,75.9
50,2031399,4814892,76.9
50,2032326,4814892,76.5
50,2033252,4814892,76.5


In [3]:
convert_date = function(df, collection_year, variable){
    # """
    # Creating a function to convert day of the year into a date with the month, day, and year
    # :param (input): dataframe, year the data was collected, variable name
    # :output: 1 df containing the coordinates, value, date of collection, variable name
    # """
    
    new_df = df %>%
        mutate(Month = month(as_date(DOY)), Day = mday(as_date(DOY)), Year = collection_year,
           Date = format(make_date(month = Month, day = Day, year = Year), format = "%m/%d/%Y"),
           # adding in the variable's name
           VariableName = variable) %>%
        select(-c('DOY', "Month", "Day", "Year")) 

    return(new_df)
    }

# calling fn
converted_tmax_2020_df = convert_date(ohio_tmax_2020_df, 2020, 'Maximum Temperature')
converted_tmax_2021_df = convert_date(ohio_tmax_2021_df, 2021, 'Maximum Temperature')
converted_tmax_2022_df = convert_date(ohio_tmax_2022_df, 2022, 'Maximum Temperature')
converted_HI_2020_df = convert_date(ohio_HI_2020_df, 2020, 'Heat Index')
converted_HI_2021_df = convert_date(ohio_HI_2021_df, 2021, 'Heat Index')
converted_HI_2022_df = convert_date(ohio_HI_2022_df, 2022, 'Heat Index')
converted_tmin_2020_df = convert_date(ohio_tmin_2020_df, 2020, 'Minimum Temperature')
converted_tmin_2021_df = convert_date(ohio_tmin_2021_df, 2021, 'Minimum Temperature')
converted_tmin_2022_df = convert_date(ohio_tmin_2022_df, 2022, 'Minimum Temperature')
# moving around these col names here to combine dfs below
converted_PM_2020_df = convert_date(ohio_PM_2020_df, 2020, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2021_df = convert_date(ohio_PM_2021_df, 2021, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2022_df = convert_date(ohio_PM_2022_df, 2022, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)

head(converted_tmax_2020_df)

x,y,Value,Date,VariableName
<dbl>,<dbl>,<dbl>,<chr>,<chr>
2032326,4815818,76.1,02/20/2020,Maximum Temperature
2033252,4815818,76.1,02/20/2020,Maximum Temperature
2030473,4814892,75.9,02/20/2020,Maximum Temperature
2031399,4814892,76.9,02/20/2020,Maximum Temperature
2032326,4814892,76.5,02/20/2020,Maximum Temperature
2033252,4814892,76.5,02/20/2020,Maximum Temperature


In [4]:
convert_coordinates = function(df){
    # """
    # Creating a function to convert UTM coordinates to latitude and longitude
    # :param (input): dataframe
    # :output: dataframe
    # """

    coordinates_df = df %>%
        # zone 14 is for Ohio
        mutate(Latitude = utm2lonlat(easting = x, northing = y, zone = 14)[[1]],
              Longitude = utm2lonlat(easting = x, northing = y, zone = 14)[[2]]) %>%
        select(-c('x', 'y')) 
    
    return(coordinates_df)
}

# calling fn
final_tmax_2020_df = convert_coordinates(converted_tmax_2020_df)
final_tmax_2021_df = convert_coordinates(converted_tmax_2021_df)
final_tmax_2022_df = convert_coordinates(converted_tmax_2022_df)
final_HI_2020_df = convert_coordinates(converted_HI_2020_df)
final_HI_2021_df = convert_coordinates(converted_HI_2021_df)
final_HI_2022_df = convert_coordinates(converted_HI_2022_df)
final_tmin_2020_df = convert_coordinates(converted_tmin_2020_df)
final_tmin_2021_df = convert_coordinates(converted_tmin_2021_df)
final_tmin_2022_df = convert_coordinates(converted_tmin_2022_df)

head(final_tmax_2020_df)

Value,Date,VariableName,Latitude,Longitude
<dbl>,<chr>,<chr>,<dbl>,<dbl>
76.1,02/20/2020,Maximum Temperature,-80.53294,41.98001
76.1,02/20/2020,Maximum Temperature,-80.52234,41.97824
75.9,02/20/2020,Maximum Temperature,-80.55653,41.97562
76.9,02/20/2020,Maximum Temperature,-80.54592,41.97386
76.5,02/20/2020,Maximum Temperature,-80.53531,41.97209
76.5,02/20/2020,Maximum Temperature,-80.52471,41.97032


Now breaking these data up into the dataframes that they will be stored in for the database. We'll have separate dataframes for each year. 

In [5]:
twenty20_df = rbind(converted_PM_2020_df, final_HI_2020_df, final_tmax_2020_df, final_tmin_2020_df)
twenty21_df = rbind(converted_PM_2021_df, final_HI_2021_df, final_tmax_2021_df, final_tmin_2021_df)
twenty22_df = rbind(converted_PM_2022_df, final_HI_2022_df, final_tmax_2022_df, final_tmin_2022_df)

In [13]:
split_dfs = function(df){
    # """
    # Creating a function to add in georaphic id and variable ids
    # :param (input): dataframe
    # :output: a list of dataframes for geography, variable, value
    # """

    new_df = df %>%
        mutate(Num = 1:n(), GeographyWID = paste0('GEOW', Num), ValueID = paste0('Val', Num),
              VariableID = ifelse(VariableName == "PM2.5", 'PM',
                                      ifelse(VariableName == "Maximum Temperature",'MaxT',
                                          ifelse(VariableName == "Minimum Temperature", 'MinT',
                                              ifelse(VariableName == "Heat Index", 'HI', NA))))) %>%
        select(-Num)

    # creating the geography weather df
    geo_df = unique(new_df[,c(6,4,5)])

    # creating the variable df
    var_df = unique(new_df[,c(8,3)]) %>%
        # adding in units
        mutate(DataUnitName = ifelse(VariableName == "PM2.5", 'mcg/m^3',
                                      ifelse(VariableName == "Maximum Temperature",'Fahrenheit',
                                          ifelse(VariableName == "Minimum Temperature", 'Fahrenheit',
                                              ifelse(VariableName == "Heat Index", 'Fahrenheit', NA)))))

    # value df
    value_df = new_df[,c(7,8,6,2,1)] 

    return(list(geo_df, var_df, value_df))
}

# calling fn
split_twenty20_dfs = split_dfs(twenty20_df)
split_twenty21_dfs = split_dfs(twenty21_df)
split_twenty22_dfs = split_dfs(twenty22_df)

In [14]:
# viewing
head(split_twenty20_dfs[[1]])
head(split_twenty20_dfs[[2]])
head(split_twenty20_dfs[[3]])

GeographyWID,Latitude,Longitude
<chr>,<dbl>,<dbl>
GEOW1,41.9661,-80.585
GEOW2,41.9661,-80.555
GEOW3,41.9661,-80.525
GEOW4,41.9361,-80.675
GEOW5,41.9361,-80.645
GEOW6,41.9361,-80.615


VariableID,VariableName,DataUnitName
<chr>,<chr>,<chr>
PM,PM2.5,mcg/m^3
HI,Heat Index,Fahrenheit
MaxT,Maximum Temperature,Fahrenheit
MinT,Minimum Temperature,Fahrenheit


ValueID,VariableID,GeographyWID,Date,Value
<chr>,<chr>,<chr>,<chr>,<dbl>
Val1,PM,GEOW1,07/08/2020,12.0
Val2,PM,GEOW2,07/08/2020,12.0
Val3,PM,GEOW3,07/08/2020,11.9
Val4,PM,GEOW4,07/08/2020,12.2
Val5,PM,GEOW5,07/08/2020,12.2
Val6,PM,GEOW6,07/08/2020,12.1


Adding in additional geography data like census tract, county name, and the FIPS code to the geography dataframes.

In [22]:
# download shapefile from the tigris (census package)
ohio_county_shapefile = data.frame(tracts(state = "Ohio", year = 2020)[,c(2,4,5,11,12)]) %>%
    rename(FIPS = GEOID, CensusTract = NAME, Latitude = INTPTLON, Longitude = INTPTLAT) %>%
    select(-geometry) %>%
    mutate(Latitude = as.numeric(Latitude), Longitude = as.numeric(Longitude))

head(ohio_county_shapefile)

,COUNTYFP,FIPS,CensusTract,Longitude,Latitude
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,153,39153533102,5331.02,41.24444,-81.41674
2,153,39153533200,5332,41.21832,-81.41640
3,029,39029951600,9516,40.72806,-80.58597
4,093,39093070600,706,41.37921,-82.09238
5,093,39093097100,971,41.12235,-82.26356
6,093,39093010200,102,41.50711,-82.01240


Finding the min and max latitude and longitude coordinates for each census tract in order to figure out what census tract each pair of coordinates reside in.

In [23]:
length(unique(ohio_county_shapefile$CensusTract))

[1] 2514

Ohio actually has 2,952 census tracts, meaning we're missing about 400 from the tracts that were downloaded from the `tigris` package.

In [25]:
min_max_coord_df = ohio_county_shapefile %>%
    group_by(CensusTract) %>%
    summarize(min_long = min(Longitude), max_long = max(Longitude),
            min_lat = min(Latitude), max_lat = max(Latitude))

head(min_max_coord_df)

CensusTract,min_long,max_long,min_lat,max_lat
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,39.35855,41.43686,-84.55411,-81.92987
1.01,41.92455,41.92455,-80.59875,-80.59875
1.02,41.95816,41.95816,-80.55897,-80.55897
1.03,41.94151,41.94151,-80.56908,-80.56908
1.10,40.05899,40.05899,-83.01276,-83.01276
1.20,40.05105,40.05105,-83.02368,-83.02368


In [18]:
geography_2020_df = head(split_twenty20_dfs[[1]]) %>%
    mutate(DataUnitName = ifelse(VariableName == "PM2.5", 'mcg/m^3',
                                      ifelse(VariableName == "Maximum Temperature",'Fahrenheit',
                                          ifelse(VariableName == "Minimum Temperature", 'Fahrenheit',
                                              ifelse(VariableName == "Heat Index", 'Fahrenheit', NA)))))
head(geography_2020_df

GeographyWID,Latitude,Longitude
<chr>,<dbl>,<dbl>
GEOW1,41.9661,-80.585
GEOW2,41.9661,-80.555
GEOW3,41.9661,-80.525
GEOW4,41.9361,-80.675
GEOW5,41.9361,-80.645
GEOW6,41.9361,-80.615
GEOW7,41.9361,-80.585
GEOW8,41.9361,-80.555
GEOW9,41.9361,-80.525
